In [28]:
import numpy as np 
import pandas as pd
import gensim 
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("~/Desktop/R_js/data/pro_doc_sampled.csv")
display(df.head())
display(df.shape)

,Unnamed: 0,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name
0,0,2.0,1,2017,"hour lunch, friendly co-workers.",cowork friendli hour lunch friendli_cowork hou...,"['cowork', 'friendli', 'hour', 'lunch', 'frien...",9,3.0,2.0,2.0,1.0,3.0,1,Walmart
1,1,3.0,1,2015,"benefits, Medical, dental, myshare, 401k, stocks",benefit dental medic myshar stock,"['benefit', 'dental', 'medic', 'myshar', 'stock']",5,3.0,4.0,4.0,3.0,3.0,1,Walmart
2,2,2.0,0,2012,discount card . schedule 3 on 3 off,card discount schedul discount_card,"['card', 'discount', 'schedul', 'discount_card']",4,1.0,4.0,3.0,1.0,3.0,1,Walmart
3,3,4.0,0,2017,good break lengths and plenty of hours,break hour length plenti plenti_hour,"['break', 'hour', 'length', 'plenti', 'plenti_...",5,4.0,3.0,3.0,4.0,4.0,1,Walmart
4,4,1.0,1,2016,My pay was good and my schedule allowed me to ...,allow job pay schedul pay_schedul,"['allow', 'job', 'pay', 'schedul', 'pay_schedul']",5,3.0,3.0,1.0,1.0,1.0,1,Walmart


(33624, 15)

In [29]:
reviews = df.Review_Text.tolist()
print(reviews[:5])

['hour lunch, friendly co-workers.', 'benefits, Medical, dental, myshare, 401k, stocks', 'discount card . schedule 3 on 3 off', 'good break lengths and plenty of hours', 'My pay was good and my schedule allowed me to work my other jobs']


In [30]:
# Data Clean
import re
import string
import spacy
from nltk.corpus import stopwords 

# stop_words = spacy.lang.en.stop_words.STOP_WORDS
stop_words = list(set(stopwords.words('english')))
stop_words.append("myshare")

def remove_url(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)


def remove_at(text):
    return re.sub('@[^\s]+', '', text)


def remove_hash(text):
    return text.replace("#", "")


def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))


def remove_letter_repitition(text):
    return re.sub(r'\d+','', text)

def remove_numbers(text):
    return re.sub(r'(.)\1+', r'\1\1', text)

def remove_stopwords(text): 
    return " ".join([word for word in text.split() if word not in stop_words])

def remove_alphabets(text):
    return ' '.join( [w for w in input.split() if len(w)>1] )



def clean_data(text):
    text = text.lower()
    text = remove_url(text)
    text = remove_at(text)
    text = remove_hash(text)
    text = remove_punct(text)
    text = remove_letter_repitition(text)
    text = remove_stopwords(text)
    return text


if __name__ == '__main__':
    print(clean_data("What the !! up holy graill aeroplane a"))

holy graill aeroplane


In [31]:
tok_reviews = [nltk.word_tokenize(clean_data(review)) for review in reviews]
print(tok_reviews[:5])

[['hour', 'lunch', 'friendly', 'coworkers'], ['benefits', 'medical', 'dental', 'k', 'stocks'], ['discount', 'card', 'schedule'], ['good', 'break', 'lengths', 'plenty', 'hours'], ['pay', 'good', 'schedule', 'allowed', 'work', 'jobs']]


In [32]:
df['Tknz_Texts1'] = tok_reviews

In [33]:
df.head()

,Unnamed: 0,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name,Tknz_Texts1
0,0,2.0,1,2017,"hour lunch, friendly co-workers.",cowork friendli hour lunch friendli_cowork hou...,"['cowork', 'friendli', 'hour', 'lunch', 'frien...",9,3.0,2.0,2.0,1.0,3.0,1,Walmart,"[hour, lunch, friendly, coworkers]"
1,1,3.0,1,2015,"benefits, Medical, dental, myshare, 401k, stocks",benefit dental medic myshar stock,"['benefit', 'dental', 'medic', 'myshar', 'stock']",5,3.0,4.0,4.0,3.0,3.0,1,Walmart,"[benefits, medical, dental, k, stocks]"
2,2,2.0,0,2012,discount card . schedule 3 on 3 off,card discount schedul discount_card,"['card', 'discount', 'schedul', 'discount_card']",4,1.0,4.0,3.0,1.0,3.0,1,Walmart,"[discount, card, schedule]"
3,3,4.0,0,2017,good break lengths and plenty of hours,break hour length plenti plenti_hour,"['break', 'hour', 'length', 'plenti', 'plenti_...",5,4.0,3.0,3.0,4.0,4.0,1,Walmart,"[good, break, lengths, plenty, hours]"
4,4,1.0,1,2016,My pay was good and my schedule allowed me to ...,allow job pay schedul pay_schedul,"['allow', 'job', 'pay', 'schedul', 'pay_schedul']",5,3.0,3.0,1.0,1.0,1.0,1,Walmart,"[pay, good, schedule, allowed, work, jobs]"


In [42]:
num_features = 300  # Word vector dimensionality
min_word_count = 3 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 3        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(tok_reviews,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model....


In [43]:
# model = gensim.models.Word2Vec.load('model')

In [44]:
model.most_similar('home')

[('remote', 0.9186325073242188),
 ('arrangement', 0.9027407169342041),
 ('remotely', 0.8952901363372803),
 ('close', 0.8744305968284607),
 ('flexibility', 0.8671191930770874),
 ('ability', 0.8670336008071899),
 ('schedule', 0.8513563871383667),
 ('schedules', 0.8484823703765869),
 ('compressed', 0.8381692171096802),
 ('arrangements', 0.8299863338470459)]

In [45]:
print(model['friendly'])
print(model["friendly"].shape)

[ 0.03377626 -0.01819777 -0.01754741 -0.04460302 -0.01786285  0.06017548
  0.04980672  0.01641134  0.06767074  0.01402638  0.06533532 -0.00359033
  0.08907183 -0.05239775  0.00646814 -0.00147883 -0.06832053  0.01136162
 -0.02340017  0.01045318  0.05648714  0.01041333 -0.06664628 -0.08295068
  0.02064369  0.04487361  0.07686536 -0.0808038  -0.04493472 -0.04452851
 -0.00741632  0.05945476  0.01566405  0.01923566 -0.03968652  0.02115248
  0.0373999  -0.02503275 -0.06903453 -0.02365107 -0.04626999 -0.00290489
 -0.06158046 -0.0166582  -0.08856279  0.03584373  0.03796189  0.03374622
  0.04027409  0.04542462 -0.02814871  0.03083507  0.14288293 -0.01458513
  0.09228542  0.11613233 -0.03448578 -0.08415332 -0.02876488 -0.13764526
  0.01512539  0.03431089  0.01553581 -0.00954222  0.01192343  0.02318529
 -0.01879531  0.00570917  0.01411439 -0.00880946  0.01481309 -0.05939163
 -0.06698534  0.05418802  0.06306943 -0.15331163 -0.08073255  0.07379588
  0.0051581  -0.06208843  0.01275224  0.07656609  0

In [38]:
print(model)

Word2Vec(vocab=3324, size=250, alpha=0.025)


In [46]:
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [47]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [48]:
# # Calculating average feature vector for training set
# clean_train_reviews = []
# for review in train['review']:
#     clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))

num_features = 300
trainDataVecs = getAvgFeatureVecs(tok_reviews, model, num_features)

Review 0 of 33624
Review 1000 of 33624
Review 2000 of 33624
Review 3000 of 33624
Review 4000 of 33624
Review 5000 of 33624
Review 6000 of 33624
Review 7000 of 33624
Review 8000 of 33624
Review 9000 of 33624
Review 10000 of 33624
Review 11000 of 33624
Review 12000 of 33624
Review 13000 of 33624
Review 14000 of 33624
Review 15000 of 33624
Review 16000 of 33624
Review 17000 of 33624
Review 18000 of 33624
Review 19000 of 33624
Review 20000 of 33624
Review 21000 of 33624
Review 22000 of 33624
Review 23000 of 33624
Review 24000 of 33624
Review 25000 of 33624
Review 26000 of 33624
Review 27000 of 33624
Review 28000 of 33624
Review 29000 of 33624
Review 30000 of 33624
Review 31000 of 33624
Review 32000 of 33624
Review 33000 of 33624


In [50]:
X = trainDataVecs
print(X.shape)

(33624, 300)


In [51]:
print(X[:5])

[[ 0.04121789 -0.01396845  0.02696263 ... -0.02370166  0.02093637
   0.10853527]
 [ 0.08462768  0.04050964  0.05484878 ...  0.02259805  0.00673683
   0.04267374]
 [ 0.04135107 -0.00094406  0.05054475 ... -0.00558681  0.02162572
   0.07136192]
 [ 0.03397564 -0.03176266  0.04216586 ... -0.01376421  0.06493688
   0.08409881]
 [ 0.02312144 -0.04160699  0.02693193 ...  0.0321114   0.05140239
   0.08364717]]


In [53]:
import math
X = [xs for xs in X if not any(math.isnan(x) for x in xs)]

In [54]:
# KMeans Clustering
from sklearn.cluster import KMeans

num_cluster = 8 
kmeans = KMeans(n_clusters=num_cluster).fit(X)

for k in range(8):
    wv = kmeans.cluster_centers_[k]
    print(f"Cluster {k}: ")
    similar_words = model.most_similar(positive=[wv], topn=30)
    print([x[0] for x in similar_words])
    print()

Cluster 0: 
['matter', 'uniform', 'freebies', 'nothing', 'downtown', 'r', 'reports', 'either', 'left', 'banks', 'bankers', 'except', 'quite', 'counts', 'mechanics', 'entertainment', 'basically', 'printing', 'desks', 'treat', 'offices', 'socials', 'et', 'associate', 'none', 'dependable', 'apart', 'older', 'youre', 'winning']

Cluster 1: 
['college', 'generous', 'sabbaticals', 'union', 'including', 'coverage', 'stocks', 'hourly', 'wage', 'packages', 'wages', 'plans', 'reimbursment', 'ok', 'comapny', 'rate', 'beneifts', 'premiums', 'amount', 'optical', 'representation', 'commission', 'raises', 'huddles', 'eligible', 'fulltime', 'eye', 'yearly', 'pension', 'benifits']

Cluster 2: 
['dinners', 'breakroom', 'occasionally', 'occasions', 'soft', 'special', 'hot', 'chocolate', 'sometime', 'often', 'catered', 'parties', 'beverages', 'raffles', 'prizes', 'cup', 'fridays', 'water', 'starbucks', 'fresh', 'fruits', 'tickets', 'tea', 'snack', 'periodic', 'pizza', 'trucks', 'sodas', 'gatorade', 'juice

# Trying out LDA on different pre-processed text

In [55]:
data_px = df.Tknz_Texts1.tolist()

In [56]:
# Create Dictionary
id2word = corpora.Dictionary(data_px)
print(len(id2word))
print(id2word)

# Create Corpus
texts = data_px

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])
print(corpus[:5])

9177
Dictionary(9177 unique tokens: ['coworkers', 'friendly', 'hour', 'lunch', 'benefits']...)
[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(9, 1), (10, 1), (11, 1)], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(11, 1), (13, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


In [57]:
mallet_path = '~/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word, iterations=500, 
                                            random_seed=42)

In [58]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_px, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.31431422041183954


In [59]:
##mallet model to lda model conversion

optimal_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [60]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint
# %%time
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.093720 -0.137420       1        1  10.047078
3     -0.084768 -0.030520       2        1  10.036695
4     -0.043812  0.040849       3        1  10.021787
7     -0.040290 -0.211077       4        1  10.019655
9      0.006931 -0.295722       5        1  10.018794
5      0.199610 -0.101327       6        1  10.007428
6     -0.249440  0.190179       7        1   9.975923
0      0.279014  0.180607       8        1   9.968948
2      0.277220  0.167663       9        1   9.952363
8     -0.250745  0.196767      10        1   9.951330, topic_info=     Category         Freq           Term        Total  loglift  logprob
20    Default  5415.000000           work  5415.000000  30.0000  30.0000
13    Default  5129.000000           good  5129.000000  29.0000  29.0000
4     Default  9034.000000       benefits  9034.000000  28.0000  28.0000
19    Default  5589.000000            pay  5589.000000  27.0000  27.0000
53    Default  7119.000000          great  7119.000000  26.0000  26.0000
77    Default  5452.000000           free  5452.000000  25.0000  25.0000
83    Default  2497.000000           time  2497.000000  24.0000  24.0000
25    Default  3018.000000         people  3018.000000  23.0000  23.0000
3     Default  2220.000000          lunch  2220.000000  22.0000  22.0000
199   Default  2253.000000    environment  2253.000000  21.0000  21.0000
14    Default  2080.000000          hours  2080.000000  20.0000  20.0000
29    Default  1892.000000        lunches  1892.000000  19.0000  19.0000
34    Default  1724.000000       flexible  1724.000000  18.0000  18.0000
41    Default  1647.000000           paid  1647.000000  17.0000  17.0000
0     Default  1540.000000      coworkers  1540.000000  16.0000  16.0000
138   Default  1437.000000         health  1437.000000  15.0000  15.0000
46    Default  1383.000000      excellent  1383.000000  14.0000  14.0000
244   Default  1265.000000           team  1265.000000  13.0000  13.0000
38    Default  1256.000000         breaks  1256.000000  12.0000  12.0000
127   Default  1591.000000     management  1591.000000  11.0000  11.0000
1     Default  1214.000000       friendly  1214.000000  10.0000  10.0000
205   Default  1206.000000           nice  1206.000000   9.0000   9.0000
107   Default  1143.000000        balance  1143.000000   8.0000   8.0000
79    Default  1131.000000           food  1131.000000   7.0000   7.0000
501   Default  1112.000000  opportunities  1112.000000   6.0000   6.0000
403   Default  1103.000000       training  1103.000000   5.0000   5.0000
27    Default  1112.000000            fun  1112.000000   4.0000   4.0000
432   Default  1300.000000        company  1300.000000   3.0000   3.0000
198   Default  1052.000000      employees  1052.000000   2.0000   2.0000
485   Default  1066.000000           home  1066.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
156   Topic10   323.254528     incentives   323.254528   2.3075  -4.0937
39    Topic10   277.541766         minute   277.541766   2.3075  -4.2462
954   Topic10   262.304179       provided   263.369775   2.3034  -4.3027
1613  Topic10   166.525060     membership   166.525060   2.3075  -4.7570
82    Topic10   161.083064         shifts   161.083064   2.3075  -4.7902
2616  Topic10   133.873087     commission   133.873087   2.3075  -4.9753
28    Topic10   120.812298             hr   120.812298   2.3075  -5.0779
227   Topic10   115.370303         awards   115.370303   2.3075  -5.1240
262   Topic10   189.381441            min   190.437889   2.3019  -4.6284
139   Topic10   107.751509          short   107.751509   2.3075  -5.1923
148   Topic10   107.751509      quarterly   107.751509   2.3075  -5.1923
497   Topic10   103.397913          based   103.397913   2.3075  -5.2336
1191  Topic10   102.309514          games   102.309514   2.3075  -5.2442
500   Topic10   101.2211